In [ ]:
%pip install rpy2

In [ ]:
%env R_HOME=C:\Program Files\R\R-4.3.3

In [ ]:
import numpy as np

import rpy2.robjects as robjects
from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr

r = robjects.r
numpy2ri.activate()

class Model(object):
    """
    R Model Loader

    Attributes
    ----------
    model : R object
    """

    def __init__(self):
        self.model = None

    def load(self, path):
        model_rds_path = "{}.rds".format(path)
        model_dep_path = "{}.dep".format(path)

        self.model = r.readRDS(model_rds_path)

        with open(model_dep_path, "rt") as f:
            model_dep_list = [importr(dep.strip())
                              for dep in f.readlines()
                              if dep.strip()!='']

        return self

    def predict(self, X):
        """
        Perform classification on samples in X.
        
        Parameters
        ----------
        X : array, shape (n_samples, n_features)
        Returns
        -------
        pred_probs : array, shape (n_samples, probs)
        """

        if self.model is None:
            raise Exception("There is no Model")
        
        if type(X) is not np.ndarray:
            X = np.array(X)

        pred = r.predict(self.model, X)
        print("pred:", pred)
        probs = r.attr(pred, "probabilities")
        print("Probs:", probs)

        return np.array(probs)


In [ ]:
import numpy as np

# Constants
MODEL_PATH = "./svm_model/artifact"

# Example Input
X = np.array([[5.1, 3.5, 1.4, 0.2], # setosa
              [5.9, 3.0, 5.1, 1.8]])  # virginica

# Example Run
model = Model().load(MODEL_PATH)
print(model)
pred = model.predict(X)

# Example output
pred.shape

In [ ]:
# Load training and test datasets
from sys import version_info
import sklearn
import mlflow.pyfunc


PYTHON_VERSION = "{major}.{minor}.{micro}".format(major=version_info.major,
                                                  minor=version_info.minor,
                                                  micro=version_info.micro)

# Train and save an SKLearn model
r_model_path = "./svm_model/artifact"

artifacts = {
    "model_rds_path" : "{}.rds".format(r_model_path),
    "model_dep_path" : "{}.dep".format(r_model_path)
}

# create wrapper
class SKLearnWrapper(mlflow.pyfunc.PythonModel):

    def load_context(self, context):

        self.model = r.readRDS(context.artifacts["model_rds_path"])

        with open(context.artifacts["model_rds_path"], "rt") as f:
            model_dep_list = [importr(dep.strip())
                              for dep in f.readlines()
                              if dep.strip()!='']

        return self
        
    
    def predict(self, model, data):
        if self.model is None:
            raise Exception("There is no Model")
        
        if type(X) is not np.ndarray:
            X = np.array(X)

        pred = r.predict(self.model, X)
        print("pred:", pred)
        probs = r.attr(pred, "probabilities")
        print("Probs:", probs)

        return np.array(probs)

        # return self.sklearn_model.predict(data)

In [ ]:
conda_env = {
    'channels': ['defaults'],
    'dependencies': [
      'python={}'.format(PYTHON_VERSION),
      'pip',
      {
        'pip': [
          'mlflow',
          'rpy2'.format(sklearn.__version__),
        ],
      },
    ],
    'name': 'sklearn_env'
}


In [ ]:
mlflow_pyfunc_model_path = "r_mlflow_pyfunc_svm"
mlflow.pyfunc.save_model(path=mlflow_pyfunc_model_path, python_model=SKLearnWrapper(), conda_env=conda_env, artifacts=artifacts)


In [ ]:
# Import the necessary libraries
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()
# # Get a handle to the workspace
import os 

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id = "781b03e7-6eb7-4506-bab8-cf3a0d89b1d4",
    resource_group_name = "antonslutsky-rg",
    workspace_name = "gpu-workspace",
)



In [ ]:

# Provide the model details, including the
# path to the model files, if you've stored them locally.
mlflow_model = Model(
    path=mlflow_pyfunc_model_path,
    type=AssetTypes.MLFLOW_MODEL,
    name=mlflow_pyfunc_model_path,
    description="MLflow Model created from local files.",
)

# Register the model
ml_client.models.create_or_update(mlflow_model)

In [ ]:
import uuid

# Create a unique name for the endpoint
online_endpoint_name = "r-svm-endpoint-" + str(uuid.uuid4())[:8]

from azure.ai.ml.entities import ManagedOnlineEndpoint

# define an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
)

In [42]:
online_endpoint_name

'r-svm-endpoint-baccbec5'

In [ ]:
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [ ]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

In [ ]:
from azure.ai.ml.entities import ManagedOnlineDeployment

# Choose the latest version of our registered model for deployment
model = ml_client.models.get(name=mlflow_pyfunc_model_path, label="latest")

# define an online deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1,
    environment={"image" : "23317781110c4faf976319f2fd34b6cf.azurecr.io/azureml/azureml_50f4dcf01428c2ff477c0c35653b323d"},
    environment_variables={
        "MLFLOW_MODEL_FOLDER": "model"
        }
)

In [ ]:
# create the online deployment
blue_deployment = ml_client.online_deployments.begin_create_or_update(
    blue_deployment
).result()

# blue deployment takes 100% traffic
# expect the deployment to take approximately 8 to 10 minutes.
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [ ]:
%%writefile blue_deployment.yml 
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: blue
endpoint_name: r-svm-endpoint-baccbec5
model:
  path: ./r_mlflow_pyfunc_svm
environment: 
  image: mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest
instance_type: Standard_DS3_v2
instance_count: 1